In [1]:
# ────────────────────────────────────────────────
# Sales Data Analysis Project – OOP Style
# Run this in Google Colab
# ────────────────────────────────────────────────

# ==================================================
# 1. Imports
# ==================================================
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from google.colab import files
import io

pd.set_option('display.float_format', '{:,.2f}'.format)
pd.set_option('display.max_columns', None)


# ==================================================
# 2. Classes
# ==================================================

class SalesDataLoader:
    """Responsible for uploading and loading the CSV file"""

    def __init__(self):
        self.df = None

    def upload_file(self):
        print("Please upload your sales CSV file...")
        uploaded = files.upload()
        if not uploaded:
            raise ValueError("No file was uploaded.")

        # Get the first (and hopefully only) uploaded file
        filename = next(iter(uploaded))
        print(f"Uploaded file: {filename}")

        # Read into pandas
        self.df = pd.read_csv(io.BytesIO(uploaded[filename]), encoding='iso-8859-1')
        print(f"Loaded {len(self.df):,} rows and {len(self.df.columns)} columns.")
        return self.df

    def get_data(self):
        if self.df is None:
            raise ValueError("No data loaded. Call upload_file() first.")
        return self.df


class SalesDataCleaner:
    """Handles data cleaning steps"""

    def __init__(self, df):
        self.df = df.copy()

    def remove_missing_order_ids(self):
        before = len(self.df)
        self.df = self.df.dropna(subset=['OrderID'])
        print(f"Removed {before - len(self.df):,} rows with missing OrderID.")
        return self

    def rename_columns(self):
        rename_dict = {
            'City.1': 'SuppCity',
            'Country.1': 'SuppCountry',
            'Phone.1': 'SuppPhone'
        }
        self.df.rename(columns=rename_dict, inplace=True)
        return self

    def convert_dates_and_ids(self):
        # Date
        if 'OrderDate' in self.df.columns:
            self.df['OrderDate'] = pd.to_datetime(self.df['OrderDate'])

        # ID columns to int
        id_cols = [col for col in self.df.columns if 'ID' in col or 'OrderNumber' in col]
        for col in id_cols:
            self.df[col] = pd.to_numeric(self.df[col], errors='coerce').fillna(0).astype(int)
        return self

    def drop_duplicates(self):
        before = len(self.df)
        self.df.drop_duplicates(subset='OrderID', inplace=True)
        print(f"Removed {before - len(self.df):,} duplicate OrderIDs.")
        return self

    def clean_column_names(self):
        self.df.columns = self.df.columns.str.strip().str.lower()
        return self

    def remove_outliers_totalamount(self, upper_multiplier=1.5):
        q1 = self.df['totalamount'].quantile(0.25)
        q3 = self.df['totalamount'].quantile(0.75)
        iqr = q3 - q1
        upper = q3 + upper_multiplier * iqr

        before = len(self.df)
        self.df = self.df[self.df['totalamount'] <= upper].copy()
        print(f"Removed {before - len(self.df):,} rows as totalamount outliers (upper bound = {upper:,.0f})")
        return self

    def clean(self):
        steps = [
            self.remove_missing_order_ids,
            self.rename_columns,
            self.convert_dates_and_ids,
            self.drop_duplicates,
            self.clean_column_names,
            self.remove_outliers_totalamount
        ]

        for step in steps:
            step()

        print("\nCleaning completed. Final shape:", self.df.shape)
        return self.df


class SalesDataAnalyzer:
    """Main analysis and visualization class"""

    def __init__(self, df):
        self.df = df.copy()
        # Quick helper columns
        self.df['fullname'] = self.df['firstname'] + ' ' + self.df['lastname']

    def top_countries_by_revenue(self, top_n=10):
        return (
            self.df.groupby('country', as_index=False)['totalamount']
            .sum()
            .sort_values('totalamount', ascending=False)
            .head(top_n)
        )

    def top_cities_by_revenue(self, top_n=10):
        return (
            self.df.groupby('city', as_index=False)['totalamount']
            .sum()
            .sort_values('totalamount', ascending=False)
            .head(top_n)
        )

    def monthly_sales_trend(self):
        monthly = (
            self.df.set_index('orderdate')['totalamount']
            .resample('ME')
            .sum()
            .reset_index()
        )
        monthly['orderdate'] = monthly['orderdate'].dt.strftime('%Y-%m')
        return monthly

    def plot_top_countries_bar(self):
        data = self.top_countries_by_revenue(12)
        fig = px.bar(
            data, x='country', y='totalamount',
            title='Top Countries by Total Revenue',
            labels={'totalamount': 'Revenue', 'country': 'Country'},
            text_auto=True
        )
        fig.update_traces(textposition='outside')
        fig.show()

    def plot_monthly_trend_line(self):
        monthly = self.monthly_sales_trend()
        fig = px.line(
            monthly, x='orderdate', y='totalamount',
            title='Monthly Sales Trend',
            labels={'orderdate': 'Month', 'totalamount': 'Revenue'},
            markers=True
        )
        fig.show()

    def plot_top_customers_bar(self, top_n=10):
        cust = (
            self.df.groupby('fullname', as_index=False)['totalamount']
            .sum()
            .sort_values('totalamount', ascending=False)
            .head(top_n)
        )
        fig = px.bar(
            cust, x='fullname', y='totalamount',
            title=f'Top {top_n} Customers by Total Spent',
            text_auto=True
        )
        fig.update_traces(textposition='outside')
        fig.show()

    def run_all_visuals(self):
        print("Generating main visualizations...")
        self.plot_top_countries_bar()
        self.plot_monthly_trend_line()
        self.plot_top_customers_bar()


# ==================================================
# Main Execution Flow
# ==================================================

if __name__ == '__main__':
    print("=== Sales Data Analysis – OOP Version ===\n")

    # 1. Load data
    loader = SalesDataLoader()
    raw_df = loader.upload_file()

    # 2. Clean data
    cleaner = SalesDataCleaner(raw_df)
    clean_df = cleaner.clean()

    # 3. Analyze & Visualize
    analyzer = SalesDataAnalyzer(clean_df)
    analyzer.run_all_visuals()

    # Optional: quick summary tables
    print("\nTop 8 countries by revenue:")
    print(analyzer.top_countries_by_revenue(8))

    print("\nTop 8 cities by revenue:")
    print(analyzer.top_cities_by_revenue(8))

=== Sales Data Analysis – OOP Version ===

Please upload your sales CSV file...


Saving all_data.csv to all_data.csv
Uploaded file: all_data.csv
Loaded 2,161 rows and 22 columns.
Removed 6 rows with missing OrderID.
Removed 1,325 duplicate OrderIDs.
Removed 56 rows as totalamount outliers (upper bound = 4,352)

Cleaning completed. Final shape: (774, 22)
Generating main visualizations...



Top 8 countries by revenue:
      country  totalamount
8     Germany   158,488.53
19        USA   138,453.74
3      Brazil    86,877.28
7      France    73,123.06
20  Venezuela    60,814.89
18         UK    55,941.51
16     Sweden    52,996.45
1     Austria    51,189.53

Top 8 cities by revenue:
              city  totalamount
35          London    49,795.21
20       Cunewalde    48,324.10
9            Boise    40,080.65
52  Rio de Janeiro    38,189.18
25            Graz    35,671.18
56       Sao Paulo    33,505.17
18            Cork    32,012.85
2      Albuquerque    30,370.30
